![title](img/dask.png)

#### **Dask provides advanced parallelism for analytics, enabling performance at scale for the tools you love**

#### **Dask is composed of two parts:**

    1. Dynamic task scheduling optimized for computation. This is similar to Airflow, Luigi, Celery, or Make, but optimized for interactive computational workloads.
    2. “Big Data” collections like parallel arrays, dataframes, and lists that extend common interfaces like NumPy, Pandas, or Python iterators to larger-than-memory or distributed environments. These parallel collections run on top of dynamic task schedulers.
    
[detailed information here](https://docs.dask.org/en/latest/)

#### **DASK with all its features:**
    • Built in Python
    • Scales properly from single laptops to 1000-node clusters
    • Leverages and interops with existing Python APIs as much as possible
    • Adheres to (Tim Peters') "Zen of Python" (https://www.python.org/dev/peps/pep-0020/) ... especially these elements:
        ◦ Explicit is better than implicit.
        ◦ Simple is better than complex.
        ◦ Complex is better than complicated.
        ◦ Readability counts. [ed: that goes for docs, too!]
        ◦ Special cases aren't special enough to break the rules.
        ◦ Although practicality beats purity.
        ◦ In the face of ambiguity, refuse the temptation to guess.
        ◦ If the implementation is hard to explain, it's a bad idea.
        ◦ If the implementation is easy to explain, it may be a good idea.
    • While we're borrowing inspiration, it Dask embodies one of Perl's slogans, making easy things easy and hard things possible
        ◦ Specifically, it supports common data-parallel abstractions like Pandas and Numpy
        ◦ But also allows scheduling arbitary custom computation that doesn't fit a preset mold
   

    


#### **Dask emphasizes the following virtues:**


    • Familiar: Provides parallelized NumPy array and Pandas DataFrame objects
    • Flexible: Provides a task scheduling interface for more custom workloads and integration with other projects.
    • Native: Enables distributed computing in pure Python with access to the PyData stack.
    • Fast: Operates with low overhead, low latency, and minimal serialization necessary for fast numerical algorithms
    • Scales up: Runs resiliently on clusters with 1000s of cores
    • Scales down: Trivial to set up and run on a laptop in a single process
    • Responsive: Designed with interactive computing in mind, it provides rapid feedback and diagnostics to aid humans.

# **DASK DISTRIBUTED** 

![title](img/distributed-overview.svg)

## **DASK DISTRIBUTED SCHEDULER**

In [1]:
import dask
from dask.distributed import Client
import numpy as np


In [2]:
# client = Client(n_workers=2, threads_per_worker=1, memory_limit='1GB')
client = Client()
client


/home/koustav/.local/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44777 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:45041 Dashboard: http://127.0.0.1:44777/status,Cluster Workers: 4 Cores: 12 Memory: 8.18 GB


#### **WORKING OF THE DISTRIBUTED SCHEDULER**

To begin, the futures interface (derived from the built-in `concurrent.futures`) allow map-reduce like functionality. We can submit individual functions for evaluation with one set of inputs, or evaluated over a sequence of inputs with `submit()` and `map()`. Notice that the call returns immediately, giving one or more futures, whose status begins as "pending" and later becomes "finished". There is no blocking of the local Python session.

In [ ]:
def func(x):
    return x + 1

fut = client.submit(func, 9)
fut

In [ ]:
fut

In [3]:
x = client.submit(np.random.random, (10000, 10000))
type(x)

distributed.client.Future

In [4]:
x.result().shape #slow as Gather the numpy array to the local process, access the .shape attribute

(10000, 10000)

In [5]:
client.submit(lambda a: a.shape, x).result() #fast as  Send a lambda function up to the cluster to compute the shape

(10000, 10000)

Dask limitations https://distributed.dask.org/en/latest/limitations.html

In [ ]:
#DATA LOCALITY 

## Task Submission
## Data scatter
## User control
## Specify workers with Compute/Persist

https://distributed.dask.org/en/latest/manage-computation.html and managing memory too





In [6]:
import dask.dataframe as dd


ddf = dd.read_csv('/home/koustav/Documents/DataframeWar/Data/yellow_tripdata_2020-01.csv', blocksize=12e6)

In [7]:
ddf

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
npartitions=50,,,,,,,,,,,,,,,,,,
,int64,object,object,int64,float64,int64,object,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [8]:
ddf.map_partitions(type).compute()


ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+-----------------+---------+----------+
| Column          | Found   | Expected |
+-----------------+---------+----------+
| RatecodeID      | float64 | int64    |
| VendorID        | float64 | int64    |
| passenger_count | float64 | int64    |
| payment_type    | float64 | int64    |
+-----------------+---------+----------+

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'RatecodeID': 'float64',
       'VendorID': 'float64',
       'passenger_count': 'float64',
       'payment_type': 'float64'}

to the call to `read_csv`/`read_table`.

Alternatively, provide `assume_missing=True` to interpret
all unspecified integer columns as floats.

In [ ]:
ddf.head()


In [ ]:
ddf['VendorID'].unique()

In [9]:
client.shutdown()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


### **Build scalable custom applications** 

    
    • Dynamically submit tasks
    • Millisecond latencies
    • Coroutines with async/await
    • high level coordination
        ◦ Locks
        ◦ Queues
        ◦ Pub/Sub
        ◦ Actors
    • Integrates with any asyncio/tornado